In [15]:
import os
import json
from typing import TypedDict, List
from dotenv import load_dotenv

from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langgraph.graph import StateGraph, END

load_dotenv()

True

In [16]:
print(os.getcwd())

c:\Users\SiddheshLihe\langhchain_skills\module_1\langgraph


In [17]:
MEMORY_FILE = "memory.json"

class ChatState(TypedDict):
    messages: List[BaseMessage]
    user_name: str

In [18]:
#loading memory
def load_memory():
    if os.path.exists(MEMORY_FILE):
        with open(MEMORY_FILE, "r") as f:
            return json.load(f)
    return {}

def save_memory(memory):
    with open(MEMORY_FILE, "w") as f:
        json.dump(memory, f)

In [19]:
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    api_key=os.getenv("GROQ_API_KEY"),
)

In [20]:
#Node: Extract User Name
def extract_user_info(state: ChatState) -> ChatState:
    user_msg = state["messages"][-1].content.lower()

    memory = load_memory()

    if "my name is" in user_msg:
        name = user_msg.split("my name is")[-1].strip()
        memory["user_name"] = name
        save_memory(memory)
        print(" Saved name to memory:", name)

    return state

In [21]:
#Node: LLM Response
def generate_response(state: ChatState) -> ChatState:
    memory = load_memory()

    system_prompt = "You are a helpful AI assistant."

    if "user_name" in memory:
        system_prompt += f" The user's name is {memory['user_name']}."

    full_messages = [HumanMessage(content=system_prompt)] + state["messages"]

    response = llm.invoke(full_messages)

    print(" LLM CALLED")
    print(" Input Messages:", full_messages)
    print(" Output:", response.content)

    return {
        "messages": state["messages"] + [response],
        "user_name": memory.get("user_name", "")
    }


In [22]:
builder = StateGraph(ChatState)

builder.add_node("Extract_user",extract_user_info)
builder.add_node("Generate_response",generate_response)

builder.set_entry_point("Extract_user")

builder.add_edge("Extract_user", "Generate_response")
builder.add_edge("Generate_response", END)

graph = builder.compile()

In [23]:
#chat_loop

In [24]:
state = {
    "messages": [],
    "user_name": ""
}

In [25]:
from IPython.display import display
import ipywidgets as widgets
from langchain_core.messages import HumanMessage

# Initialize state
state = {"messages": []}

# Create widgets
text = widgets.Text(placeholder="Type your message here")
send = widgets.Button(description="Send")
out = widgets.Output()

def on_send(b):
    user_input = text.value.strip()
    if not user_input:
        return
    
    # Exit condition
    if user_input.lower() in ["exit", "quit", "bye"]:
        with out:
            print("\n👋 Chat ended.")
        text.disabled = True
        send.disabled = True
        return

    # Show user input
    with out:
        print("\nYou:", user_input)

    # Append to state
    state["messages"].append(HumanMessage(content=user_input))

    # Invoke graph
    new_state = graph.invoke(state)

    # Show AI response
    with out:
        print("AI:", new_state["messages"][-1].content)

    # Update state
    state.update(new_state)

    # Clear text box
    text.value = ""

# Bind button click
send.on_click(on_send)

# Display UI
display(text, send, out)


Text(value='', placeholder='Type your message here')

Button(description='Send', style=ButtonStyle())

Output()

In [26]:
print("termloop")

termloop
